In [2]:
search_for = 137
start_from = 0
threads = 10
thread = 7
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.00480341911315918
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 354865621
type: [1, 1, 1, 1, 137] 137
cases of this type: 352275361
100000 14.652795932975321
200000 28.880411911390343
300000 34.208209337404284
400000 33.05105251991514
500000 32.58980527034626
600000 33.33475064185878
700000 34.9794315629907
800000 33.10736657050764
900000 32.687405118758285
1000000 32.54498577637293
1100000 32.40536532607772
1200000 32.45106034361302
1300000 15.821595697934104
1400000 24.15116299545899
1500000 32.34958839264955
1600000 31.52947734438199
1700000 31.87244323567336
1800000 31.922730549305022
1900000 32.44219647417287
2000000 33.66077465810363
2100000 32.340061669838086
2200000 31.9055938399703
2300000 32.36243216248199
2400000 33.192437591068874
2500000 32.81552853413321
2600000 13.179034796371859
2700000 28.43384482618213
2800000 40.4705159876784
2900000 41.533898286387945
3000000 40.87070344753

29800000 31.488234327342607
29900000 32.486194919143145
30000000 33.0234850595708
30100000 33.00653260066179
30200000 32.47679908529655
30300000 33.29795958276235
30400000 34.02302852661862
30500000 33.56292248573992
30600000 32.27285912930468
30700000 12.782526608217541
30800000 34.53104841816546
30900000 34.052953159415196
31000000 33.32575666264866
31100000 20.10296126134146
31200000 24.51230524681993
31300000 33.3407068176318
31400000 33.069726270550035
31500000 32.796596151062346
31600000 32.33427813070793
31700000 33.74827654315347
31800000 32.34480236870071
31900000 31.510543332706877
32000000 31.58533350548844
32100000 30.236552439375767
32200000 7.522126407898971
32300000 31.37487870607795
32400000 30.98675735680854
32500000 31.242017408742342
32600000 31.92501275678055
32700000 32.42063909192295
32800000 31.014841007445547
32900000 32.7926498065624
33000000 33.08879187691289
33100000 32.80729646950312
33200000 20.381198895224056
33300000 25.04693173698074
33400000 33.76209100

59700000 23.873327995990287
59800000 29.65579506909675
59900000 28.275054498572086
60000000 28.800920529538562
60100000 27.42899980670677
60200000 27.356141726527543
60300000 27.215110512802838
60400000 23.108574121967116
60500000 11.81318128228692
60600000 27.099001343990945
60700000 27.604589260418276
60800000 27.309939368204624
60900000 28.138769191429187
61000000 28.748177993107344
61100000 28.221380385656353
61200000 29.301574444789818
61300000 13.770902517984526
61400000 27.80774943983584
61500000 30.24652704679819
61600000 29.245452601050783
61700000 30.774394242961186
61800000 29.560423264723447
61900000 29.809337400205475
62000000 29.651192007209975
62100000 30.106158520570578
62200000 13.307176471432673
62300000 25.616958589920724
62400000 28.708887888749125
62500000 27.76116355489289
62600000 28.666603841258926
62700000 27.36088509678252
62800000 26.68188013175601
62900000 26.0596103355563
63000000 16.256113146011522
63100000 18.356102074846543
63200000 26.493195112084056
63

89400000 8.740949896917764
89500000 26.67805556907408
89600000 25.77005332554882
89700000 26.484605938571594
89800000 26.72398041522163
89900000 26.994996845278777
90000000 27.573206361998256
90100000 26.21793353777877
90200000 26.896799236518728
90300000 26.932612888984323
90400000 26.30909194522871
90500000 26.046050363705476
90600000 26.25334033597333
90700000 10.11481291837414
90800000 22.70660430416674
90900000 24.684576019968624
91000000 23.53952779668721
91100000 24.074440689623362
91200000 24.31499659527499
91300000 11.497845584605145
91400000 19.04643976272565
91500000 24.499460673330614
91600000 23.872584217294385
91700000 23.990340455144764
91800000 24.208876983544275
91900000 17.178058664577954
92000000 18.709080992364544
92100000 25.684669680779702
92200000 25.636566389428825
92300000 26.998678423741175
92400000 26.47930267837235
92500000 27.22948780638686
92600000 26.26357422899189
92700000 26.7452478886228
92800000 26.571127717916745
92900000 26.71617026485895
93000000 2

118500000 23.449873185811782
118600000 24.131416205878864
118700000 24.921451659232584
118800000 21.69580659151359
118900000 22.916512194188083
119000000 21.01513897843097
119100000 22.533054439828692
119200000 6.572457756659048
119300000 22.769229254901134
119400000 21.165468983250406
119500000 21.748765830021338
119600000 7.164329982650714
119700000 19.884422246616385
119800000 21.2179483852964
119900000 22.84532265003214
120000000 11.461424468103102
120100000 17.60353381942515
120200000 22.42279989567901
120300000 20.72762097148008
120400000 21.48209133280834
120500000 22.911104233612093
120600000 22.687875677802026
120700000 24.06227324459072
120800000 22.86283177794305
120900000 23.394335633391854
121000000 23.835022807682673
121100000 23.606572415286024
121200000 23.290412566730822
121300000 23.440147361612176
121400000 22.258334981503822
121500000 21.923501559935602
121600000 21.019413860861373
121700000 22.355375674182447
121800000 6.544973284220787
121900000 21.124433939429903

...

233100000 9.158094578862102
233200000 5.707376081466932
233300000 11.25721858576326
233400000 11.003063106869215
233500000 11.559610887118845
233600000 11.572547181719248
233700000 11.92592151210212
233800000 12.069299740516769
233900000 12.19073050014652
234000000 12.27276766712653
234100000 12.171501692844052
234200000 12.238699449793952
234300000 12.179193579088109
234400000 11.934108464174932
234500000 11.123219027159545
234600000 11.143617293571607
234700000 10.805308983009683
234800000 11.116313616362484
234900000 4.0138608049450655
235000000 11.177529388375298
235100000 11.039318730850548
235200000 10.936370416002624
235300000 4.642907211029435
235400000 9.227978808085492
235500000 10.84093636258721
235600000 10.682553322729154
235700000 8.1260482531795
235800000 6.640635436961098
235900000 10.583266090329525
236000000 10.994354898552126
236100000 11.419793497366847
236200000 11.524774885132581
236300000 11.706787819324143
236400000 12.113190381888097
236500000 11.66224540586853

262100000 9.083528809535514
262200000 9.682209495385232
262300000 8.666483281478483
262400000 9.242019007821531
262500000 9.086623514789077
262600000 9.095203169834933
262700000 8.856616868839517
262800000 8.631090325399326
262900000 7.512638461402618
263000000 4.51496231649684
263100000 8.106684927838646
263200000 8.145411317129213
263300000 8.008199074611314
263400000 8.080800981436301
263500000 8.489678015130604
263600000 2.428574028528403
263700000 8.237818046841518
263800000 8.485249655240368
263900000 8.674668339375811
264000000 8.3080912218217
264100000 8.393103482956748
264200000 5.616587280445221
264300000 5.745177073705946
264400000 8.632615960917423
264500000 8.82491400117301
264600000 8.884489716720818
264700000 8.964579341520746
264800000 9.286789786592152
264900000 9.428269974067959
265000000 9.225396376227081
265100000 8.905738636595435
265200000 9.168099991555716
265300000 8.759244861990581
265400000 8.65732617835383
265500000 4.463841146266417
265600000 7.0991087359450

291400000 5.953257899381361
291500000 5.7669263792723315
291600000 5.779186713382053
291700000 5.777998960358343
291800000 5.861992974150765
291900000 1.1010697288615996
292000000 5.954172896689874
292100000 5.75477044651481
292200000 5.74782660858774
292300000 5.712976021064174
292400000 6.015108220444446
292500000 5.919082827949441
292600000 6.114836433580466
292700000 3.794968406970009
292800000 4.471673434120573
292900000 6.268188959191239
293000000 6.164346394935228
293100000 6.271984671409267
293200000 6.221244797337379
293300000 6.2596848694134275
293400000 6.211517601809981
293500000 6.146693293662091
293600000 2.116738357402476
293700000 6.692898461320271
293800000 5.901200760876525
293900000 5.702272316311108
294000000 5.770284633536284
294100000 5.574986111227414
294200000 5.410416610163315
294300000 5.495682044741084
294400000 4.284992537278342
294500000 2.788640979463559
294600000 5.410549759010389
294700000 5.365453050377525
294800000 5.474175554580381
294900000 6.1722152

320600000 3.394161917391312
320700000 3.5423602143367203
320800000 3.349916136859435
320900000 3.4124715897299738
321000000 3.446357265013205
321100000 3.402292521274481
321200000 1.9024030209115308
321300000 2.725390523052951
321400000 3.5467521028260527
321500000 3.3897590001258395
321600000 3.181951052681823
321700000 1.477925613473886
321800000 3.3911886825265998
321900000 3.3378835317013267
322000000 3.498742427620498
322100000 3.2562483188379847
322200000 3.1816024865299033
322300000 3.3155921545212883
322400000 3.2448572968090605
322500000 3.1603130670859065
322600000 3.257544907214695
322700000 2.027506622064101
322800000 1.9383790502250464
322900000 3.1671644297903225
323000000 3.373689327651229
323100000 3.1578943341293395
323200000 3.3727835397289634
323300000 3.1703878868452597
323400000 3.2275809725560074
323500000 3.4054979037012703
323600000 3.2149441699766244
323700000 3.4053898757982246
323800000 1.4960417554017484
323900000 3.002247585863622
324000000 3.39928049664893

349200000 0.6381293902617449
349300000 0.6236024118625443
349400000 0.6000724190294021
349500000 0.5940977256035752
349600000 0.5717209186943472
349700000 0.26027004146751365
349800000 0.2688516532100598
349900000 0.43217694203819235
350000000 0.4206254173691511
350100000 0.4035555965781139
350200000 0.4026704699931125
350300000 0.3896404909569575
350400000 0.3980291291692588
350500000 0.37249648759434817
350600000 0.3571090944118427
350700000 0.35301695505715347
350800000 0.3463445884112974
350900000 0.3352347931115263
351000000 0.1779745573761238
351100000 0.22718032214143077
351200000 0.2939205397509449
351300000 0.308549736203097
351400000 0.2826943157153189
351500000 0.26530698331756525
351600000 0.27371289391672876
351700000 0.26652194390268324
351800000 0.25293810439394115
351900000 0.2505226174452749
352000000 0.252534664187332
352100000 0.22349582554870578
352200000 0.23164827237752808
best so far: 0
type: [1, 1, 1, 137, 1] 137
cases of this type: 2571353
352300000 0.086758986